# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [15]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [17]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
High-efficiency quantum rectenna for millimeter wave and terahertz technologies	Talk	APS2023	APS March Meeting 2023	2023-03-10	Las Vegas, Nevada	https://meetings.aps.org/Meeting/MAR23/Session/Y43.8	Talk at APS March Meeting 2023. A related preprint is at https://arxiv.org/abs/2211.17219
Fundamental bound on topological gap	Talk	APS2024	APS March Meeting 2024	2024-03-08	Minneapolis, Minnesota	https://meetings.aps.org/Meeting/MAR24/Session/Y17.12	Talk at APS March Meeting 2024. A related publication is https://journals.aps.org/prx/abstract/10.1103/PhysRevX.14.011052
Quantum geometry, Optical property, and Energy gap	Seminar	AS2024Feb	Academia Sinica	2024-02-22	Taipei, Taiwan		Seminar at Academia Sinica at Taipei, Taiwan. Hosted by Prof. Hsin Lin.
Quantum geometry, Optical property, and Energy gap	Seminar	NTU2024Feb	National Taiwan University	2024-02-23	Taipei, Taiwan	https://www.phys.ncts.ntu.edu.tw/en/act/TG3_1act/%5BNCTS-Phys

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [14]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,High-efficiency quantum rectenna for millimete...,Talk,APS2023,APS March Meeting 2023,2023-03-10,"Las Vegas, Nevada",https://meetings.aps.org/Meeting/MAR23/Session...,Talk at APS March Meeting 2023. A related prep...
1,Fundamental bound on topological gap,Talk,APS2024,APS March Meeting 2024,2024-03-08,"Minneapolis, Minnesota",https://meetings.aps.org/Meeting/MAR24/Session...,Talk at APS March Meeting 2024. A related publ...
2,"Quantum geometry, Optical property, and Energy...",Seminar,AS2024Feb,Academia Sinica,2024-02-22,"Taipei, Taiwan",NaN,"Seminar at Academia Sinica at Taipei, Taiwan. ..."
3,"Quantum geometry, Optical property, and Energy...",Seminar,NTU2024Feb,National Taiwan University,2024-02-23,"Taipei, Taiwan",https://www.phys.ncts.ntu.edu.tw/en/act/TG3_1a...,Seminar at National Taiwan University at Taipe...
4,"Quantum geometry, Optical property, and Energy...",Seminar,NCKU2024Feb,National Cheng Kung University,2024-02-27,"Tainan, Taiwan",https://www.phys.ncku.edu.tw/en/colloquia-deta...,"Seminar at Cheng Kung University at Tainan, Ta..."
5,Fundamental gap bound in insulators,Seminar,HarvardKids2024,Harvard CMT Kids' Seminar,2024-03-26,"Cambridge, Massachusetts",http://cmt.harvard.edu/kidseminar.html,Harvard CMT Kids’ Seminar.
6,Fundamental gap bound in insulators,Seminar,UCLA2024Apr,Group meeting of Narang group at UCLA,2024-04-03,Online,NaN,Informal seminar at Narang group at UCLA.
7,Quantum weight,Seminar,DelftU2024Apr,Speakers’ Corner Talk,2024-04-05,Online,https://virtualscienceforum.org/speakers-corner/,Hosted by Virtual Science Forum. The recording...
8,Fundamental bound on topological gap,Seminar,ETH2024Apr,ETH Zurich,2024-04-10,"ETH Zurich, Switzerland",https://www.lifescience.net/events/69335/yugo-...,Seminar at ETH Zurich. Hosted by Prof. Ataç İm...
9,Fundamental gap bound in insulators,Talk,NQCM2024May,NQCM2024,2024-05-25,"University of Tokyo, Japan",https://sites.google.com/view/nqcm2024/home?au...,Invited talk for a workshop at University of T...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [18]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [16]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [20]:
!ls ../_talks

2023-03-10-APS2023.md	       2024-04-05-DelftU2024Apr.md
2024-02-22-AS2024Feb.md        2024-04-10-ETH2024Apr.md
2024-02-23-NTU2024Feb.md       2024-05-25-NQCM2024May.md
2024-02-27-NCKU2024Feb.md      2024-05-28-Fujimoto2024May.md
2024-03-08-APS2024.md	       2024-05-29-Emergent2024May.md
2024-03-26-HarvardKids2024.md  2024-07-18-QuantumDesigner2024Jul.md
2024-04-03-UCLA2024Apr.md


In [19]:
!cat ../_talks/2023-03-10-APS2023.md

---
title: "High-efficiency quantum rectenna for millimeter wave and terahertz technologies"
collection: talks
type: "Talk"
permalink: /talks/2023-03-10-APS2023
venue: "APS March Meeting 2023"
date: 2023-03-10
location: "Las Vegas, Nevada"
---

[More information here](https://meetings.aps.org/Meeting/MAR23/Session/Y43.8)

Talk at APS March Meeting 2023. A related preprint is at https://arxiv.org/abs/2211.17219
